`Benodigde packages:`<br>
`pip install protobuf==3.19.4 gtfs-realtime-bindings protobuf_to_dict protobuf3-to-dict==0.1.5 simplejson pandas geopandas folium urllib3 libprotobuf`<br>

`Maar op Windows doet hij het alleen als:`<br>
`conda install protobuf`<br>

`compiler protoc downloaden:`<br>
`https://github.com/protocolbuffers/protobuf/releases/tag/v3.19.4`<br>

`Python bestanden genereren:`<br>
`protoc --python_out=. *.proto`<br>

`protc.exe en de genegereerde python bestanden in dezelfde map als dit notebook`<br>

`Nog niet gelukt een omgeving te maken vanuit scratch waarin het werkt.`<br>

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from datetime import datetime

from GTFS import GTFS

In [ ]:
gtfs = GTFS()
gtfs.update_static()
gtfs.update_realtime()

In [ ]:
gtfs.update_realtime()

# Create time departure board train

In [ ]:
station = 'Arnhem Zuid'
df = gtfs.get_train_departures_at(station)

def print_train_departures(df):
    print("Station {} ({})".format(df.iloc[0]['stop_name'], df.iloc[0]['station_id']))
    print("Time   Delay   Platform         Trein         Rit     Bestemming")
    for idx, dep in df.sort_values('departure_time').head(10).iterrows():
        desc = "{:>5s}   {:>4d}      {:>5s}    {:>10s}  {:10d}     {}"
        desc = desc.format(dep['departure_time'].strftime("%H:%M"),
                          int(dep['departure_delay'] if isinstance(dep['departure_delay'], int) else 0),
                          dep['platform_code'],
                          dep['route_short_name'][0:9],
                          dep['trip_short_name'],
                          dep['trip_headsign'])
        print(desc)
    
print_train_departures(df[df.departure_time > datetime.now()])

In [ ]:
halte = 'Elderhof'
df = gtfs.get_departures_at_stop(halte)

def print_bus_departures(df):
    if len(df) > 0:
        print("Halte {} ({})".format(df.iloc[0]['stop_name'], str(df['stop_id'].unique())))
        print("Time   Delay   Platform           Bus         Rit     Bestemming")
        for idx, dep in df.sort_values('departure_time').head(10).iterrows():
            desc = "{:>5s}{:1s}  {:>4d}      {:>5s}    {:>10s}  {:10d}     {}"
            desc = desc.format(dep['departure_time'].strftime("%H:%M"),
                               '*' if dep['RT'] else ' ',
                              int(dep['departure_delay'] if isinstance(dep['departure_delay'], int) else 0),
                              str(dep['platform_code']),
                              dep['route_short_name'][0:9],
                              dep['trip_short_name'],
                              dep['trip_headsign'])
            print(desc)
    
print_bus_departures(df[df.departure_time > datetime.now()])

# Current locations (Arnhem)

In [ ]:
bbox=((5.78, 51.9), (5.98, 52.1))

In [ ]:
gdf_veh = gtfs.vehicle_positions.cx[bbox[0][0]:bbox[1][0], bbox[0][1]:bbox[1][1]] 
gdf_veh = gdf_veh.merge(gtfs.trips, on=['trip_id', 'route_id'])
gdf_veh = gdf_veh.merge(gtfs.stoptimes, left_on=['trip_id', 'current_stop_seq'], right_on=['trip_id', 'stop_sequence'])
gdf_veh = gdf_veh.merge(gtfs.stops, on='stop_id')

In [ ]:
gdf_halte = gtfs.stops.cx[bbox[0][0]:bbox[1][0], bbox[0][1]:bbox[1][1]] 

In [ ]:
# Map of Arnhem
map_osm = folium.Map(location=[51.98, 5.89], zoom_start=12)

# Popup formatter
fs = "<table><tr><td>ID</td><td>{}</td></tr>" +\
     "<tr><td></td><td>{}</td></tr>"+\
     "<tr><td>Location</td><td>{:6.4f}, {:6.4f}</td></tr></table>"
for _, veh in gdf_halte.iterrows():
    iframe = folium.IFrame(fs.format(veh['stop_id'], veh['stop_name'], 
                                     veh['stop_lat'], veh['stop_lon']))
    popup = folium.Popup(iframe, min_width=300, max_width=300)
    marker = folium.CircleMarker(location=[veh["stop_lat"], veh["stop_lon"]], 
                                 popup=popup,
                                 radius=1, color='blue')
    map_osm.add_child(marker)

fs = "<table><tr><td>ID</td><td>{}</td></tr>" +\
     "<tr><td></td><td>{}</td></tr>"+\
     "<tr><td>Lijn</td><td>{} : {}</td></tr>"+\
     "<tr><td>Halte</td><td>{}</td></tr>"+\
     "<tr><td>Location</td><td>{:6.4f}, {:6.4f}</td></tr></table>"

for _, veh in gdf_veh.iterrows():
    iframe = folium.IFrame(fs.format(veh['route_id'], veh['agency_id'], veh['route_short_name'], 
                                     veh['trip_headsign'], veh['stop_name'],
                                     veh['latitude'], veh['longitude']))
    popup = folium.Popup(iframe, min_width=300, max_width=300)
    marker = folium.CircleMarker(location=[veh["latitude"], veh["longitude"]], 
                                 popup=popup,
                                 radius=5, color='red')
    map_osm.add_child(marker)

map_osm

In [ ]:
# from folium import plugins

# map = folium.Map(location=[51.98, 5.89], zoom_start=12) #, tiles="Cartodb dark_matter")

# heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf_halte.geometry]

# heat_data
# plugins.HeatMap(heat_data, radius=20, blur=10).add_to(map)

# map

In [ ]:
from folium import plugins

map = folium.Map(location=[52.0, 5.1], zoom_start=8, width=1000, height=600, tiles="Cartodb dark_matter")

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gtfs.stops.geometry]

heat_data
# plugins.HeatMap(heat_data, radius=6, blur=5).add_to(map)
plugins.HeatMap(heat_data, radius=3, blur=1).add_to(map)

map